In [2]:
from coding import *
import numpy as np

# n=45, s=2, d=1

In [3]:
d=1
n=45
s=2
mp.dps = 25
W, fake_W, W_perp, C_1 = search_w_mp(n=n, s=s)
simulated_G = mp.randmatrix(n, d)
simulated_adv = mp.zeros(n, d)
simulated_adv[0:2,:] = -100

simulated_R = W * simulated_G + simulated_adv
random_vec = mp.randmatrix(d, 1)

R_1 = simulated_R * random_vec

E_2 = W_perp * R_1

E_matrix = mp.zeros(s, s)
E_b = mp.zeros(s, 1)

for i in range(s):
    E_matrix[i,:] = E_2[2*s-i-s-1:2*s-i-1].transpose()
    E_b[i] = E_2[2*s-i-1]

alpha=mp.lu_solve(E_matrix, E_b)
print(-1*alpha)
print("-----------------------------------------------------")

# let's generate true alpha
roots = [np.exp(2*np.pi*i*1j/n) for i in range(s)]
np_poly=P.polyfromroots(roots)
print(np_poly)
print("=====================================================")

[(0.9902680679767230413121403 + 0.1391730995353912744873454j)]
[ (-1.99026806789378627786873 - 0.1391730996145090608984356j)]
-----------------------------------------------------
[ 0.99026807+0.1391731j -1.99026807-0.1391731j  1.00000000+0.j       ]


# n=45, s=4, d=1

In [6]:
d=1
n=45
s=4
mp.dps = 100
W, fake_W, W_perp, C_1 = search_w_mp(n=n, s=s)
simulated_G = mp.randmatrix(n, d)
simulated_adv = mp.zeros(n, d)
simulated_adv[0:2,:] = -100

simulated_R = W * simulated_G + simulated_adv
random_vec = mp.randmatrix(d, 1)

R_1 = simulated_R * random_vec

E_2 = W_perp * R_1

E_matrix = mp.zeros(s, s)
E_b = mp.zeros(s, 1)

for i in range(s):
    E_matrix[i,:] = E_2[2*s-i-s-1:2*s-i-1].transpose()
    E_b[i] = E_2[2*s-i-1]

alpha=mp.lu_solve(E_matrix, E_b)
print(-1*alpha)
print("-----------------------------------------------------")

# let's generate true alpha
roots = [np.exp(2*np.pi*i*1j/n) for i in range(s)]
np_poly=P.polyfromroots(roots)
print(np_poly)
print("=====================================================")

[  (0.9184535681958170663555515 + 0.1291831839979683493125996j)]
[ (-0.4892494405489177560098457 + 0.1494915595063512135687547j)]
[ (-0.8212291612382202905781961 - 0.2267494186147808579040751j)]
[(-0.6079749652451070191488674 - 0.05192532422769364548398816j)]
-----------------------------------------------------
[ 0.66913061+0.74314483j -3.19676860-2.32258834j  5.39271322+2.40099062j
 -3.86507522-0.8215471j   1.00000000+0.j        ]
